<h1>Import Necessary Libraries</h1>

In [ ]:

import os
import numpy as np
from utils import *
import keras
import segmentation_models_3D as sm

<h1>Load model checkpoint from local directory</h1>

In [ ]:
BACKBONE = 'resnet50'
preprocess_input = sm.get_preprocessing(BACKBONE)
IMG_SIZE=320

model = sm.Unet(
    BACKBONE, 
    input_shape=(IMG_SIZE, IMG_SIZE, IMG_SIZE, 1), 
    encoder_weights=None, 
    classes=1
)

#load model checkpoint
model.load_weights('darkvol_unet0_86.4.weights.h5') 

#model = keras.models.load_model('darlvol_fold0_86.3.keras')

<h1>Read sample_IDs from "test_data" folder</h1>

In [ ]:
# Directory path
testing_data = os.path.join(
    os.getcwd(), "test_data"
)  # 'path to your training data'

# List to store the names of subfolders (sample IDs)
sample_ids = []

# Iterate over the items in the directory
for item in os.listdir(testing_data):
    item_path = os.path.join(testing_data, item)
    if os.path.isdir(item_path):
        sample_ids.append(item)



In [ ]:
#loop over each test sample
for idx in range(len(sample_ids)):
    # Choosing one of the samples
    sample_id = sample_ids[idx]
    
    # Path to the subfolder/sample
    sample_path = os.path.join(testing_data, sample_id)
    
    # List and print all files in the sample subfolder
    files = os.listdir(sample_path)
    print('processing sample', idx)
    
    # Iterate over the files and load the .npy files.
    for file in files:
        if file.startswith("seismicCubes_") and file.endswith(".npy"):
            seismic = np.load(os.path.join(sample_path, file), allow_pickle=True)
           
            seismic_rescaled = rescale_volume(seismic, low=2, high=98).astype(np.uint8)
            fault = []
            for i in range(4): #each seismic sample was divided into 4 parts
                z = i*320 
                image = np.zeros((320,320,320), dtype=np.float32)
                img_data = seismic_rescaled[0:300, 0:300, z:min(z+320, 1259)]/255.0
                _, _, dz = img_data.shape
                image[10:310, 10:310, 0:dz] = img_data

                #transformation for TTA
                image2 = image[::-1, :, :]
                image3 = image[:, ::-1, :]
                image4 = image[:, :, ::-1]
                image5 = image[::-1, ::-1, ::-1]

                #get prediction
                image = np.reshape( preprocess_input(image), (1,320,320,320,1))
                image2 = np.reshape( preprocess_input(image2), (1,320,320,320,1))
                image3 = np.reshape( preprocess_input(image3), (1,320,320,320,1))
                image4 = np.reshape( preprocess_input(image4), (1,320,320,320,1))
                image5 = np.reshape( preprocess_input(image5), (1,320,320,320,1))

                #apply 5xTTA
                r1 = model.predict(image, verbose=0)[0,:,:,:,0]
                r2 = model.predict(image2, verbose=0)[0,:,:,:,0]
                r3 = model.predict(image3, verbose=0)[0,:,:,:,0]
                r4 = model.predict(image4, verbose=0)[0,:,:,:,0]
                r5 = model.predict(image5, verbose=0)[0,:,:,:,0]
                
                r1 = r1 + r2[::-1, :, :] + r3[:, ::-1, :] + r4[:, :, ::-1] + r5[::-1, ::-1, ::-1]

                fault.append(r1/5.0)

            #each prediction volumes are of shape 320x320x320 and are stacked along z-axis
            fault = np.concatenate(fault, axis=2)

            #the predicted volumes contain 10px paddings in x and y axes around the actual fault
            fault = fault[10:310, 10:310, 0:1259]  #actual fault volume
            fault = (fault > 0.45).astype(np.uint8)

            
            # this line creates the submission .npz file and populates it with sample IDs and their fault coordinates
            create_submission(sample_id, fault, "submission_TTA.npz", append=True)            
      

